## Export Feature Service
>Download data from an ArcGIS Online/Portal Hosted Feature Service

In [2]:
import arcpy
import os
import time
import zipfile
from arcgis.gis import GIS
from getpass import getpass
from zipfile import ZipFile
from zipfile import ZipInfo

In [3]:
# Directory to download exported feature service data
downloadPath = 'c://users//friendde//Downloads//UserGroupDemo//'

### If you have access to more than one Portal site (such as ArcGIS Online and ArcGIS Enterprise Portal) you can switch between the active portal or a different portal

In [ ]:
response = input(f'Do you want to use the active portal? {(arcpy.GetActivePortalURL())} [y or n] ')
if response.upper() in "YES":
    gis = GIS(arcpy.GetActivePortalURL(), username=input("Enter User Name "), password=(getpass()))
else:
    portalURL=input("Enter Portal Address")
    username=input("Enter User Name")
    gis = GIS(portalURL, username, password=getpass())

#### Verify you are connected to the intended Portal site

In [ ]:
portal_desc = arcpy.GetPortalDescription()
print(f'Portal Name {portal_desc["portalName"]}\nConnected to {portal_desc["name"]} as user {portal_desc["user"]["username"]}')

### Search and list all items hosted on the Portal
> You can limit search using query paramters such as items owned by user <br>
> Or you can search for everything the user has access to on the Portal

In [ ]:
# query items owned by logged in user
query=f'owner:{portal_desc["user"]["username"]}'
#query=f'owner:tuckerza'
# query all Feature Layer Items available to logged in user
#query=''
itemType="Feature Layer"
sortField="title"
sortOrder="asc"
# default max__items is 10
maxItems=100
searchResult = gis.content.search(query,itemType,sortField,sortOrder,maxItems)
for i in range(len(searchResult)):
    display(searchResult[i])
    downloadResult = input(f'Download {(searchResult[i].title)} [Y,N]')
    if downloadResult.upper() in "YES":
        sdItem = searchResult[i]
        print(sdItem.title, sdItem.id, sdItem.modified, sdItem.access)
        break
if downloadResult.upper() not in "YES":
    print(f'All Feature Layers hosted on {portal_desc["portalName"]} were presented')
    exit

You've selected the item to download from above <br>
Now request the Portal site to prepare the selected Item for export <br>
which will be downloaded in a later step

In [ ]:
fgdbItem = gis.content.get(sdItem.id).export(sdItem.title + '_fgdb', 'File Geodatabase', parameters=None, wait='True')
print(fgdbItem)

In [ ]:
print(sdItem.title)

gis.content.get(fgdbItem['exportItemId']).download(save_path = downloadPath)

Now download the exported Item from the Portal site <br>
> The intial output is a 1 KB file that is not a valid zipped file <br>
The ```while``` statement waits for a valid zipped file to be created

In [ ]:
zf = downloadPath + sdItem.title +'fgdb.zip'
while  zipfile.is_zipfile(zf)==False:
    print("Waiting for valid zip file")
    gis.content.get(fgdbItem['exportItemId']).download(save_path = downloadPath)
    time.sleep(30)
print(f'{zf} ready')

#### Prepare the downloaded file to be extracted from the Zip file

In [ ]:
zf_extract = ZipFile(zf)
zf_extract.extractall(path=downloadPath + sdItem.title + '_fgdb')

After extracting the Zip file show the GDB name within

In [ ]:
file_list = os.listdir(downloadPath + sdItem.title + '_fgdb')
file_list

#### Now you can use the downloaded exported Item in your ArcGIS Pro project